In [56]:
#importing necessary libraries
import pandas as pd
import numpy as np
import ast

In [57]:
# reading datasets and choosing nessasry features for learning and removing null and duplicate values
df1 = pd.read_csv('tmdb_5000_credits.csv')
df2 = pd.read_csv('tmdb_5000_movies.csv')
main = df1.merge(df2,on ='title')
main = main[['movie_id','title','overview','genres','keywords','cast','crew']]
df2.isnull().sum()
main.dropna(inplace = True)


In [58]:
#helper function to convert the list of dict to a list of str with only names
#helper function to pick onlt the top 3 ACTORS
#helper function to pick only the director


def convert(obj):
    l = []
    for i in ast.literal_eval(obj):
        l.append(str(i['name']))
    return l

def character(obj):
    l = []
    counter = 0 #only need top 3 ACTORS NAME
    for i in ast.literal_eval(obj):
        if counter !=3:
            l.append(str(i['name']))
            counter+=1
        else:
            break
    return l

def director(obj):
    l = []
    for i in ast.literal_eval(obj):
        if i['job'] == "Director":
                l.append(str(i['name']))
                break
    return l

In [59]:
#applying the fuction to each attribute
main['genres'] = main['genres'].apply(convert)
main['keywords'] = main['keywords'].apply(convert)
main['cast'] = main['cast'].apply(character)
main['crew'] = main['crew'].apply(director)
main['overview'] = main['overview'].apply(lambda x:x.split())

In [60]:
#to remove space in order to maintain uniqeness of values
main['genres'] = main['genres'].apply(lambda x :[i.replace(" ","")for i in x])
main['keywords'] = main['keywords'].apply(lambda x :[i.replace(" ","")for i in x])
main['cast'] = main['cast'].apply(lambda x :[i.replace(" ","")for i in x])
main['crew'] = main['crew'].apply(lambda x :[i.replace(" ","")for i in x])


In [61]:
#concatonating all the attributes ibto a single attribute called the tags
main['tags'] = main['genres']+main['keywords']+main['cast']+main['overview']

In [62]:
#make a new dataframe with the necessary features
#making all the tags a string
#change all the tag content to lowercase
movies = main[['movie_id','title','tags']]
movies['tags'] = movies['tags'].apply(lambda x: " ".join(x))
movies['tags'] = movies['tags'].apply(lambda x: x.lower())
movies.head()

C:\Users\viswa\AppData\Local\Temp\ipykernel_8644\3648744551.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['tags'] = movies['tags'].apply(lambda x: " ".join(x))
C:\Users\viswa\AppData\Local\Temp\ipykernel_8644\3648744551.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['tags'] = movies['tags'].apply(lambda x: x.lower())


,movie_id,title,tags
0,19995,Avatar,action adventure fantasy sciencefiction cultur...
1,285,Pirates of the Caribbean: At World's End,adventure fantasy action ocean drugabuse exoti...
2,206647,Spectre,action adventure crime spy basedonnovel secret...
3,49026,The Dark Knight Rises,action crime drama thriller dccomics crimefigh...
4,49529,John Carter,action adventure sciencefiction basedonnovel m...


In [63]:
#from nltk import porterStemmer in order to combine similar sounding words like love lovong loved etc to love
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [81]:
#helper funtion to combine similar sounding words like love lovong loved etc to love
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [82]:
#call the function to amke the changes
movies["tags"]=movies["tags"].apply(stem)
movies.tags[0]

C:\Users\viswa\AppData\Local\Temp\ipykernel_8644\1255633842.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["tags"]=movies["tags"].apply(stem)


'action adventur fantasi sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d samworthington zoesaldana sigourneyweav in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization.'

In [66]:
#import scikitlearn to use contvercorizer in order to choose thw most used 5000 words in all of the tags combined by removing
#the stops words from the english language
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000,stop_words = 'english')

In [67]:
#convert the features into an array
vectors = cv.fit_transform(movies['tags']).toarray()

In [73]:
#print the features
cv.get_feature_names_out()


array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

In [80]:
#distance of each movie with every other movie by using cosine_similarity as ot parameter
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)

array([[1.        , 0.08471737, 0.08740748, ..., 0.04499213, 0.        ,
        0.        ],
       [0.08471737, 1.        , 0.06253054, ..., 0.02414023, 0.        ,
        0.02654659],
       [0.08740748, 0.06253054, 1.        , ..., 0.02490677, 0.        ,
        0.        ],
       ...,
       [0.04499213, 0.02414023, 0.02490677, ..., 1.        , 0.03962144,
        0.04229549],
       [0.        , 0.        , 0.        , ..., 0.03962144, 1.        ,
        0.08714204],
       [0.        , 0.02654659, 0.        , ..., 0.04229549, 0.08714204,
        1.        ]])

In [104]:
#this line is used t+in the cell below the main use is to assign an index to each movie so afer it is sorted it dosn't lose its position
#later it is sorted based on the second attribute and we return oly top 5 values
#sorted(list(enumerate(similarity[0])) ,reverse = True,key = lambda x:x[1])[1:6]

In [115]:
#helper function that takes the name of the movie and return the index of the most similar 
def recom(movie):
    movie_index = movies[movies['title'] == movie].index[0]
    distances = similarity[movie_index]
    movie_list = sorted(list(enumerate(distances)) ,reverse = True,key = lambda x:x[1])[1:6]

    for i in movie_list:
        #return the name of the movie based on its index
        print(movies.iloc[i[0]].title)

In [117]:
recom('Batman Begins')

The Dark Knight
Batman
Batman
10th & Wolf
The Dark Knight Rises


In [122]:
#send the filer to pycram to display it to the website
import pickle
pickle.dump(movies.to_dict(),open('movies_dict.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))